<a href="https://colab.research.google.com/github/shinnew9/AICounselors/blob/main/data/DataTranslationAllAtOnce_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers accelerate sentencepiece
!pip install sacremoses
!python --version

### Dataset load

In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("qiuhuachuan/PsyDial-D4")

In [ ]:
import pprint

print(ds)

print(ds['train'][0].keys())  # dict_keys(['messages'])
print(ds['train'])

sample = ds['train']['messages']
pprint.pp(sample)

### Model load

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "Helsinki-NLP/opus-mt-zh-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)
model.to(device)

In [ ]:
from datasets import load_dataset

ds = load_dataset("qiuhuachuan/PsyDial-D4", split="train")  # 이제 ds는 Dataset
print(ds)

test_batch = ds.select(range(8))  # ✅ 이제 잘 됨
print(test_batch)

#### Translate Function

In [ ]:
import torch

def translate_turnwise(batch):
    sessions = batch["messages"]
    out_sessions = []

    chunk_size = 8  # 한 번에 번역할 turn 개수(조절 가능)

    for session in sessions:
        zh_list = [t["content"] for t in session]
        en_list = []

        # 여러 번 쪼개서 번역
        for start in range(0, len(zh_list), chunk_size):
            chunk = zh_list[start:start + chunk_size]

            inputs = tokenizer(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,    # turn 단위라 128이면 보통 충분
            ).to(device)

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_length=128,
                    num_beams=1,    # 🔥 beam search 끄기 (메모리 절약)
                )

            chunk_en = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            en_list.extend(chunk_en)

            # 메모리 조금이라도 정리
            del inputs, outputs
            torch.cuda.empty_cache()

        # zh/en를 role과 함께 묶기
        bi_session = []
        for t, zh, en in zip(session, zh_list, en_list):
            bi_session.append({
                "role": t["role"],
                "zh": zh,
                "en": en,
            })

        out_sessions.append(bi_session)

    return {"messages_bilingual": out_sessions}

#### Translate

In [ ]:
translated_turnwise = ds.map(
    translate_turnwise,
    batched=True,
    batch_size=1,   # 🔥 한 번에 세션 하나씩만
    num_proc=1,
    desc="Turn-wise translation (memory-safe)",
)

In [ ]:
save_path = "/content/gdrive/MyDrive/PsyDial-D4-bilingual-turnwise.jsonl"

translated_turnwise.to_json(
    save_path,
    lines=True,
    force_ascii=False
)

print("Saved:", save_path)

In [ ]:
sample = translated_turnwise[0]["messages_bilingual"][:5]
for t in sample:
    print("ROLE:", t["role"])
    print("ZH  :", t["zh"])
    print("EN  :", t["en"])
    print("----")

In [ ]:
# not working
def to_english_only(ex):
    lines = []
    for turn in ex["messages_bilingual"]:
        role = turn["role"]
        en = turn["en"]
        lines.append(f"[{role}] {en}")
    return {"dialogue_en": "\n".join(lines)}

ds_en = translated_turnwise.map(to_english_only)

ds_en.to_json(
    "/content/gdrive/MyDrive/PsyDial-D4-en-dialogues-only.jsonl",
    lines=True,
    force_ascii=False
)

# still has some chinese

In [ ]:
# not working
def drop_zh(ex):
    new_msgs = []
    for t in ex["messages_bilingual"]:
        new_msgs.append({
            "role": t["role"],
            "en": t["en"],
        })
    return {"messages_en_only": new_msgs}

ds_en_only = translated_turnwise.map(drop_zh)

ds_en_only.to_json(
    "/content/gdrive/MyDrive/PsyDial-D4-en-turnwise-only.jsonl",
    lines=True,
    force_ascii=False
)

In [ ]:
!pwd

### Dataset saving in my directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json

def load_jsonl(path):
    """
    JSONL 파일을 읽어서 Python 리스트로 반환하는 함수. 각 줄(line)은 하나의 dict가 됨.
    """
    data = []
    with open(path, "r", encoding="utf-8-sig") as f: # Changed encoding to 'utf-8-sig'
      for line in f:
          # if line.strip():  # 공백 라인 방지
          data.append(json.loads(line))
    return data

In [ ]:
import chardet

def load_jsonl_auto(path):
    # 1. detect encoding
    with open(path, "rb") as f:
        raw = f.read()
        enc = chardet.detect(raw)["encoding"]
        print("Detected encoding:", enc)

    # 2. load with detected encoding
    text = raw.decode(enc, errors="replace")
    lines = text.splitlines()

    data = [json.loads(line) for line in lines]
    return data

data = load_jsonl_auto("/content/drive/MyDrive/ProfChuah/PsyDial-D4-en-dialogues-only.jsonl")
print("Loaded:", len(data))

In [ ]:
path = "/content/drive/MyDrive/ProfChuah/PsyDial-D4-en-dialogues-only.jsonl"
data = load_jsonl(path)

# 첫 번째 세션 출력
print(data[0]["dialogue_en"])

In [ ]:
import chardet

def load_jsonl_auto(path):
    # 1. detect encoding
    with open(path, "rb") as f:
        raw = f.read()
        enc = chardet.detect(raw)["encoding"]
        print("Detected encoding:", enc)

    # 2. load with detected encoding
    text = raw.decode(enc, errors="replace")
    lines = text.splitlines()

    data = [json.loads(line) for line in lines]
    return data

data = load_jsonl_auto("/content/drive/MyDrive/ProfChuah/PsyDial-D4-en-dialogues-only.jsonl")
print("Loaded:", len(data))

In [ ]:
print(data[0]["dialogue_en"])

In [ ]:
print(data[2]["dialogue_en"])

In [ ]:
save_path = "/content/drive/MyDrive/ProfChuah/PsyDial-en-dialogues-clean.txt"

with open(save_path, "w", encoding="utf-8") as f:
    for i, item in enumerate(data):
        f.write(f"=== SESSION {i} ===\n")
        f.write(item["dialogue_en"])
        f.write("\n\n")

print("Saved to:", save_path)

In [ ]:
# 여러 세션 영어만 묶어서 하나의 TXT로 저장

save_txt = "/content/drive/MyDrive/ProfChuah/PsyDial-D4-en-dialogues-ALL.txt"

with open(save_txt, "w", encoding="utf-8") as f:
    for i, item in enumerate(data):
        f.write(f"=== SESSION {i} ===\n")
        f.write(item["dialogue_en"])
        f.write("\n\n")

In [ ]:
# not working
save_path = "/content/drive/MyDrive/ProfChuah/PsyDial-en-dialogues-list.json"

english_list = [item["dialogue_en"] for item in data]

with open(save_path, "w", encoding="utf-8") as f:
    json.dump(english_list, f, ensure_ascii=False, indent=2)

print("Saved to:", save_path)

In [ ]:
for i in range(3):
    print("=== SESSION", i, "===")
    print(data[i]["dialogue_en"][:500])   # 너무 길면 앞부분만
    print()

In [ ]:
import os

txt_path = "/content/drive/MyDrive/ProfChuah/PsyDial-D4-en-dialogues-ALL.txt"
output_dir = "/content/drive/MyDrive/ProfChuah/PsyDial-SAMPLES"
os.makedirs(output_dir, exist_ok=True)


sessions = []
current = []

with open(txt_path, encoding="utf-8") as f:
  for line in f:
    if line.startswith("=== SESSION"):
      if current:
        sessions.append("".join(current).strip())
        current = []
    else:
      current.append(line)

  # 마지막 세션 처리
  if current:
    sessions.append("".join(current).strip())

print(f"Total sessions loaded: {len(sessions)}")


# 샘플 5개 출력
for i in range(5):
  print(f"\n====== SAMPLE {i} =====\n")
  print(sessions[i])  # 너무 길면 [:800] 같은 걸로 잘라서 확인해도 됌


# save first 5 sessions
for i in range(5):
  save_path = os.path.join(output_dir, f"sample_{i}.txt")
  with open(save_path, "w", encoding="utf-8") as f:
    f.write(sessions[i])
  print(f"Saved sample {i} -> {save_path}")

In [ ]:
def extract_user_text(session_text):
    """Extract all user messages from one session."""
    user_lines = []
    for line in session_text.split("\n"):
        if line.strip().startswith("[user]"):
            user_lines.append(line.replace("[user]", "").strip())
    return " ".join(user_lines)

# Extract & summarize 5 sessions
summaries = []

for i in range(5):
    session = sessions[i]
    user_text = extract_user_text(session)

    # 간단 규칙 기반 summarizer (키워드 기반)
    summary = ""

    if any(k in user_text.lower() for k in ["roommate", "dorm", "noise", "isolate"]):
        summary += "- Roommate conflict impacting emotional well-being.\n"

    if any(k in user_text.lower() for k in ["study", "school", "pressure", "exam", "academic"]):
        summary += "- Strong academic pressure and fear of underperforming.\n"

    if any(k in user_text.lower() for k in ["parents", "family", "expect", "childhood"]):
        summary += "- High parental expectations affecting self-worth.\n"

    if any(k in user_text.lower() for k in ["anxiety", "panic", "sweat", "heartbeat"]):
        summary += "- Anxiety symptoms in social or formal situations.\n"

    if any(k in user_text.lower() for k in ["empty", "lost", "meaning", "value"]):
        summary += "- Feelings of emptiness, loss of direction, existential concerns.\n"

    if any(k in user_text.lower() for k in ["lonely", "alone", "isolated"]):
        summary += "- Persistent loneliness and lack of emotional connection.\n"

    if summary == "":
        summary = "- General emotional distress and interpersonal challenges.\n"

    summaries.append((i, summary.strip()))

# Print the summaries
for idx, summ in summaries:
    print(f"\n===== SUMMARY FOR SAMPLE {idx} =====\n{summ}\n")

In [ ]:
!pwd

### HuggingFace Upload

In [ ]:
!pip install -q "huggingface_hub[cli]"
!huggingface-cli login

In [ ]:
import os
from huggingface_hub import HfApi

api = HfApi(token=os.getenv("  "))

# Original path to the file
file_path = "/content/drive/MyDrive/ProfChuah/PsyDial-en-dialogues-clean.txt"

# Upload the single file using upload_file
api.upload_file(
    path_or_fileobj=file_path,
    path_in_repo="PsyDial-D4-en-dialogues-clean.txt", # Desired filename in the repo
    repo_id="shinnew/PsyDial4_Eng_Translated",
    repo_type="dataset",
    commit_message="Upload PsyDial-D4-en-dialogues-clean.txt"
)

print(f"File {file_path} uploaded successfully to shinnew/PsyDial4_Eng_Translated as PsyDial-D4-en-dialogues-ALL.txt")